In [1]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import ipywidgets as widgets
import seaborn as sns
import pandas as pd
import numpy as np

from plotly.subplots import make_subplots
from ipywidgets import interact

from Linear_Regression import Linear_Regression
from utils import interactive_plot

sns.set_theme()

%matplotlib widget

In [73]:
from ipywidgets import interact

def interactive_plot(X, y, arrow_length):

    # ------------ CÁLCULO DAS VARIÁVEIS ------------ #
    w0, w1 = 0, 0

    W0_ = np.linspace(-100,100,25)
    W1_ = np.linspace(-100,100,25)
    W0, W1 = np.meshgrid(W0_, W1_)

    error_surface = ((y - (W1.reshape(-1,1) * X.T + W0.reshape(-1,1)))**2).mean(axis=1).reshape(25, 25)
    dj_dw0_field = - ((y - (W1.reshape(-1,1) * X + W0.reshape(-1,1))).mean(axis=1) * 2).reshape(25,25)
    dj_dw1_field = - (((y - (W1.reshape(-1,1) * X + W0.reshape(-1,1))) * X).mean(axis=1) * 2).reshape(25,25)

    # Definindo os limites do eixo y
    max_error_surface = np.max(error_surface)
    ylim = (-max_error_surface/10, max_error_surface)
    xytext = (0, max_error_surface - max_error_surface/10)

    # ------------ PLOT ------------ #
    specs = [[{}, {}, {'type':'surface'}],
             [{}, {}, {}]]
    fig = make_subplots(rows=2, cols=3, figure=go.FigureWidget(layout=go.Layout(width=2000, height=1000)), specs=specs)

    fig.add_trace(go.Scatter(x=X, y=y, mode='markers', marker_line_width=1), row=1, col=1) # 0
    fig.add_trace(go.Scatter(x=X, mode='lines', marker_color='black'), row=1, col=1) # 1
    [fig.add_trace(go.Scatter(x=[x, x], mode='lines', line=dict(dash='dot'), marker_color='orange', showlegend=False), row=1, col=1) for x in X]
    fig.add_trace(go.Contour(x=W0_, y=W1_, z=error_surface, ncontours=25), row=1, col=2)
    fig.add_trace(go.Scatter(x=[], y=[], mode='markers', marker_color='yellow', marker=dict(size=15, line=dict(width=2, color='black'))), row=1, col=2)
    fig.add_trace(go.Surface(x=W0, y=W1, z=error_surface), row=1, col=3)
    fig.add_trace(go.Scatter3d(x=[], y=[], z=[], mode='markers', marker_color='yellow', marker=dict(size=10, line=dict(color='black'))), row=1, col=3)
    fig.add_trace(go.Scatter3d(x=[], y=[], z=[], mode='lines', marker_color='black', marker_line_width=25, line=dict(dash='dot')), row=1, col=3)

    def update(w0, w1):

        # Cálculo do MSE no ponto (w0,w1)
        y_ = w0 + w1 * X
        error = ((y - y_)**2).mean()

        # Calculando o MSE nas direções de w0 e w1
        w0_line = W0[0]
        w1_line = W1[:,0]
        error_line0 = ((y - (w1 * X.T + w0_line.reshape(-1,1)))**2).mean(axis=1)
        error_line1 = ((y - (w1_line.reshape(-1,1) * X.T + w0))**2).mean(axis=1)

        # Calculando o gradiente no ponto (w0, w1)
        dj_dw0 = - 2 * (y - y_).mean()
        dj_dw1 = - 2 * ((y - y_) * X).mean()

        # Cálculo dos parâmetros das setas
        norm0 = np.sqrt(1**2 + dj_dw0**2)
        dw0 = - norm0 * dj_dw0 / abs(dj_dw0) * arrow_length
        dj0 = - abs(dj_dw0 * norm0) * arrow_length

        norm1 = np.sqrt(1**2 + dj_dw1**2)
        dw1 = - norm1 * dj_dw1 / abs(dj_dw1) * arrow_length
        dj1 = - abs(dj_dw1 * norm1) * arrow_length

        # Atualização do Plot
        with fig.batch_update():
            fig.data[1].y = y_

            fig.data[-4].x = [w0]
            fig.data[-4].y = [w1]

            fig.data[-2].x = [w0]
            fig.data[-2].y = [w1]
            fig.data[-2].z = [error]

            fig.data[-1].x = [w0]*25
            fig.data[-1].y = w1_line
            fig.data[-1].z = error_line0
            display(error_line0)
            for data in range(2, 2+len(X)):
                fig.data[data].y = [y[data-2], y_[data-2]]
                
    interact(
        update,
        w0=widgets.FloatSlider(value=w0, min=W0.min(), max=W0.max(), step=1e-3),
        w1=widgets.FloatSlider(value=w1, min=W1.min(), max=W1.max(), step=1e-3, readout=True, readout_format='.3f')
    )

    display(fig)

In [63]:
df = pd.read_csv(r'.\Recursos\08-Linear-Regression-Models\Advertising.csv')
df['total-spend'] = df.iloc[:,:-1].sum(axis=1)
df.head()

,TV,radio,newspaper,sales,total-spend
0,230.1,37.8,69.2,22.1,337.1
1,44.5,39.3,45.1,10.4,128.9
2,17.2,45.9,69.3,9.3,132.4
3,151.5,41.3,58.5,18.5,251.3
4,180.8,10.8,58.4,12.9,250.0


In [74]:
X = df['total-spend'].head(25).values
y = df['sales'].head(25).values

interactive_plot(X, y, 3e-6)

interactive(children=(FloatSlider(value=0.0, description='w0', min=-100.0, step=0.001), FloatSlider(value=0.0,…

FigureWidget({
    'data': [{'marker': {'line': {'width': 1}},
              'mode': 'markers',
              …

### DEPOIS DE NORMALIZADO

In [ ]:
X = df['total-spend'].head(25).values
X = (X - np.mean(X)) / np.std(X)

y = df['sales'].head(25).values

interactive_plot(X, y, 2.5e-1)

In [ ]:
X = df['total-spend'].head(25).values.reshape(-1,1)
X_norm = (X - np.mean(X)) / np.std(X)

y = df['sales'].head(25).values + 20

In [ ]:
model = Linear_Regression()
model_norm = Linear_Regression()

epochs = 50
history = model.fit_gd(X, y, learning_rate=1e-6, epochs=epochs)
history_norm = model_norm.fit_gd(X_norm, y, learning_rate=1e-1, epochs=epochs)

fig = plt.figure(figsize=(16,5))
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)
ax3 = fig.add_subplot(133, sharey=ax2)

def update(epoch):
    ax1.clear()
    ax1.plot(np.log10(history['loss']), color='black', label='Custo (não normalizado)', zorder=0)
    ax1.plot(np.log10(history_norm['loss']), color='black', linestyle='--', label='Custo (normalizado)', zorder=0)
    ax1.scatter(epoch, np.log10(history['loss'][epoch]), c='yellow', ec='black', zorder=1)
    ax1.scatter(epoch, np.log10(history_norm['loss'][epoch]), c='yellow', ec='black', zorder=1)
    ax1.set_title('Curva de Aprendizado')
    ax1.set_xlabel('Época')
    ax1.set_ylabel(r'$log_{10}(\text{MSE})$')
    ax1.grid(False)
    ax1.legend()

    ax2.clear()
    ax2.scatter(X, y, color='orange', ec='black', label=r'Observado ($y$)', zorder=1)
    ax2.plot(X, history['y_pred'][epoch], color='black', linewidth=1, label=r'Calculado ($X\beta$)', zorder=0)
    ax2.set_title(f'Distribuição e Modelo Ajustado \nÉpoca: {epoch}')
    ax2.set_xlabel('X')
    ax2.set_ylabel('y')
    ax2.grid(False)
    ax2.legend()

    ax3.clear()
    ax3.scatter(X_norm, y, color='orange', ec='black', label=r'Observado ($y$)', zorder=1)
    ax3.plot(X_norm, history_norm['y_pred'][epoch], color='black', linewidth=1, label=r'Calculado ($X_{\text{norm}}\beta_{\text{norm}}$)', zorder=0)
    ax3.set_title(f'Distribuição Normalizada e Modelo Ajustado \nÉpoca: {epoch}')
    ax3.set_xlabel('X normalizado')
    ax3.set_ylabel('y')
    ax3.grid(False)
    ax3.legend()

# fig.tight_layout()
# plt.show()

interact(update, epoch=widgets.IntSlider(value=0, min=0, max=epochs-1))